In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
with open('/content/gdrive/My Drive/yelp_academic_dataset_business.json') as business_file:
  businesses = pd.read_json(business_file, lines=True)

In [0]:
vegas_businesses = businesses.loc[businesses["city"] == "Las Vegas"]


In [0]:
vegas_restaurants = vegas_businesses.loc[businesses["categories"].str.contains("Restaurants", na=False)]
vegas_restaurants = vegas_restaurants.drop_duplicates(subset="name")

4498


In [0]:
reviews = pd.DataFrame()
chunksize = 10 ** 6
for chunk in pd.read_json('/content/gdrive/My Drive/yelp_academic_dataset_review.json', lines=True, chunksize=chunksize):
  reviews = reviews.append(chunk)
  

In [0]:
vegas_reviews = reviews.loc[reviews["business_id"].isin(vegas_restaurants["business_id"])]

In [0]:
vr_samples = vegas_reviews.sample(n=500000)

865909


In [0]:
brief_reviews = vr_samples[["user_id", "business_id", "stars", "date"]]

In [0]:
train = brief_reviews.sample(frac=0.8,random_state=200)
test = brief_reviews.drop(train.index)

In [0]:
train.to_csv("/content/gdrive/My Drive/training.csv", sep='\t', encoding='utf-8', index=False)
test.to_csv("/content/gdrive/My Drive/test.csv", sep='\t', encoding='utf-8', index=False)

In [0]:
vegas_businesses_names = vegas_restaurants[["business_id", "name"]]
vegas_businesses_names.to_csv("/content/gdrive/My Drive/business_names.csv", sep='\t', encoding='utf-8', index=False)

In [0]:
reviewsByUser = {}
bus_name_dict = vegas_businesses_names.set_index("business_id").to_dict()
bus_name_dict = bus_name_dict["name"]
for index, row in brief_reviews.iterrows():
  reviewsByUser.setdefault(row["user_id"], []).append([bus_name_dict[row["business_id"]], row["stars"]])

In [0]:
import json
with open('/content/gdrive/My Drive/reviewsByUser.json', 'w') as fp:
    json.dump(reviewsByUser, fp)